In [ ]:
import pyaps3 as pa
import os
import matplotlib.pyplot as plt

# run pyaps3
data_dir = '/Users/zachkeskinen/Documents/phase-o-matic/PyAPS/tests/data'
# read geometry files
print('read ISCE geometry files: hgt/los/lat/lon.rdr')
dem_pa = pa.utils.read_data(os.path.join(data_dir, 'hgt.rdr'))
inc = pa.utils.read_data(os.path.join(data_dir, 'los.rdr'), dname='inc')
lat = pa.utils.read_data(os.path.join(data_dir, 'lat.rdr'))
lon = pa.utils.read_data(os.path.join(data_dir, 'lon.rdr'))

# calculate
print('calculate tropospheric delay from GRB files...')
print('------------------------------------------------')
grb_file1 = os.path.join(data_dir, 'ERA5/ERA5_N30_N40_E120_E140_20101017_14.grb')
grb_file2 = os.path.join(data_dir, 'ERA5/ERA5_N30_N40_E120_E140_20110117_14.grb')
obj1 = pa.PyAPS(grb_file1, dem=dem_pa, inc=inc, lat=lat, lon=lon, grib='ERA5', verb=True)
obj2 = pa.PyAPS(grb_file2, dem=dem_pa, inc=inc, lat=lat, lon=lon, grib='ERA5', verb=True)
phs = obj2.getdelay(wvl = 0.238403545) - obj1.getdelay(wvl = 0.238403545)

# regrid data to xarray
import numpy as np
import pandas as pd
import xarray as xr

import sys
sys.path.append('..')

from phase_o_matic.utils import regrid_twoD_coords

dem = pa.utils.read_data(os.path.join(data_dir, 'hgt.rdr'))
dem = xr.DataArray(dem, 
             coords = {
    "lat": (("y", "x"), lat),
    "lon": (("y", "x"), lon)
},
dims = ["y","x"])
dem = regrid_twoD_coords(dem)

inc = np.deg2rad(pa.utils.read_data(os.path.join(data_dir, 'los.rdr'), dname='inc'))
inc= xr.DataArray(inc, 
             coords = {
    "lat": (("y", "x"), lat),
    "lon": (("y", "x"), lon)
},
dims = ["y","x"])
inc = regrid_twoD_coords(inc)


pyaps_phase = xr.DataArray(phs, 
             coords = {
    "lat": (("y", "x"), lat),
    "lon": (("y", "x"), lon)
},
dims = ["y","x"])
pyaps_phase = regrid_twoD_coords(pyaps_phase)

dem_ds = dem.to_dataset(name = 'dem')
dem_ds['inc'] = inc

dates = (pd.to_datetime('2010-10-17T14:00'), pd.to_datetime('2011-01-17T14:00'))

# Phase o matic now
from shapely import geometry

from phase_o_matic.download import download_era
from phase_o_matic.preprocess import get_vapor_partial_pressure, convert_pressure_to_pascals, interpolate_to_heights, geopotential_to_geopotential_heights
from phase_o_matic.phase_delay import calculate_refractive_indexes, get_delay
eras = []
for date in dates:
    out_fp = download_era(date, out_dir = '../data/phaseo_era/', subset = geometry.box(129.25, 30.25, 132.25, 33.75), humid_param = 'specific_humidity')
    era = xr.open_dataset(out_fp)
    era = convert_pressure_to_pascals(era)
    era = get_vapor_partial_pressure(era)
    era = geopotential_to_geopotential_heights(era)
    if date == pd.to_datetime('2010-10-17T14:00'):
        ds = interpolate_to_heights(era, max_alt = obj1.hgt[-1])
    elif date == pd.to_datetime('2011-01-17T14:00'):
        ds = interpolate_to_heights(era, max_alt = obj2.hgt[-1])
    
    ds = calculate_refractive_indexes(ds)
    ds = get_delay(ds, dem_ds, wavelength = 0.238403545)
    eras.append(ds['delay'])
era = xr.concat(eras, dim = 'time')

# now plot the two
fig, axes = plt.subplots(2, 2, figsize = (12, 8))

titles = ['PyAPS (mintpy) Tropospheric delay', 'Height']
for ax, data, title in zip(axes[0, :], [phs, dem_pa], titles):
    if title == 'Height':
        im = ax.imshow(data, interpolation='nearest', vmin = 0, vmax = 1650, aspect="auto")
    else:
        im = ax.imshow(data, interpolation='nearest', aspect="auto")
    # axis format
    ax.invert_yaxis()
    ax.set_title(title)
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('meter')

phaseo = (era.isel(time = 1) - era.isel(time = 0))
phaseo.plot(ax = axes[1, 0])
(pyaps_phase - phaseo).plot(ax = axes[1, 1], vmax = 1, vmin = -1)
axes[1,0].set_title('Phase-o-matic Tropospheric Delay')
axes[1,1].set_title('Difference')

plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3)
[lvls,latlist,lonlist,gph,tmp,vpr] = pa.era.get_ecmwf(obj1.model, obj1.gfile, obj1.minlat, obj1.maxlat, obj1.minlon, obj1.maxlon, cdic = obj1.dict, humidity = obj1.humidity, verbose = False)
ax = axes[0, 0]
# era['vpr'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(era.level, era['vpr'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).data - vpr[:, 0, 0])

ax = axes[0, 1]
# era['t'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(era.level, era['t'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).data - tmp[:, 0, 0])

ax = axes[0, 2]
# (era['gph'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0])).plot(ax = ax)
ax.plot(era.level,  era['gph'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).data - gph[:, 0, 0])

[lvls,latlist,lonlist,gph,tmp,vpr] = pa.era.get_ecmwf(obj2.model, obj2.gfile, obj2.minlat, obj2.maxlat, obj2.minlon, obj2.maxlon, cdic = obj2.dict, humidity = obj2.humidity, verbose = False)
ax = axes[1, 0]
# era['vpr'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(era.level, era['vpr'].isel(time = 1).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).data - vpr[:, 0, 0])

ax = axes[1, 1]
# era['t'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(era.level, era['t'].isel(time = 1).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).data - tmp[:, 0, 0])

ax = axes[1, 2]
# (era['gph'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0])).plot(ax = ax)
ax.plot(era.level, era['gph'].isel(time = 1).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).data - gph[:, 0, 0])


plt.tight_layout()

In [ ]:
t1 = interpolate_to_heights(era.isel(time = 0).expand_dims(time = [era.isel(time = 0).time.data]), max_alt = obj1.hgt[-1])
t2 = interpolate_to_heights(era.isel(time = 1).expand_dims(time = [era.isel(time = 1).time.data]), max_alt = obj2.hgt[-1])

fig, axes = plt.subplots(2, 3)
ax = axes[0, 0]
# ds['vapor_pressure'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(obj1.hgt, obj1.Vi[0,0, :] - t1['vapor_pressure'].isel(time = 0).sel(latitude = obj1.latlist[0, 0], longitude = obj1.lonlist[0, 0]).data)
# ax.set_ylim(-2, 2)
ax = axes[0, 1]
# ds['temperature'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(obj1.hgt, obj1.Ti[0,0, :] - t1['temperature'].isel(time = 0).sel(latitude = obj1.latlist[0, 0], longitude = obj1.lonlist[0, 0]).data)
ax = axes[0, 2]
# ds['air_pressure'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(obj1.hgt, obj1.Pi[0,0, :] - t1['air_pressure'].isel(time = 0).sel(latitude = obj1.latlist[0, 0], longitude = obj1.lonlist[0, 0]))
# ax.set_ylim(-2, 2)

# ax = axes[0, 0]
# # ds['vapor_pressure'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
# ax.plot(obj2.hgt, obj2.Vi[0,0, :] - ds['vapor_pressure'].isel(time = 1).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]).data)
# # ax.set_ylim(-2, 2)
# ax = axes[0, 1]
# # ds['temperature'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
# ax.plot(obj2.hgt, obj2.Ti[0,0, :] - ds['temperature'].isel(time = 1).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]).data)

# ax = axes[0, 2]
# # ds['air_pressure'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
# ax.plot(obj2.hgt, obj2.Pi[0,0, :] - ds['air_pressure'].isel(time = 1).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]))
# # ax.set_ylim(-2, 2)

ax = axes[1, 0]
# ds['vapor_pressure'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(obj2.hgt, obj2.Vi[0,0, :] - t2['vapor_pressure'].isel(time = 0).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]).data)
# ax.set_ylim(-2, 2)
ax = axes[1, 1]
# ds['temperature'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(obj2.hgt, obj2.Ti[0,0, :] - t2['temperature'].isel(time = 0).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]).data)

ax = axes[1, 2]
# ds['air_pressure'].isel(time = 0).sel(latitude = latlist[0, 0], longitude = lonlist[0, 0]).plot(ax = ax)
ax.plot(obj2.hgt, obj2.Pi[0,0, :] - t2['air_pressure'].isel(time = 0).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]))
# ax.set_ylim(-2, 2)

In [ ]:
from scipy import integrate as intg
fig, axes = plt.subplots(3)
ax = axes[0]
[DDry,DWet] = pa.processor.PTV2del(obj1.Pi,obj1.Ti,obj1.Vi,obj1.hgt,obj1.dict,verbose=True)
ds = calculate_dry_refractivity(t1)
ax.plot(ds['N_dry'].isel(time = 0).sel(latitude = obj1.latlist[0, 0], longitude = obj1.lonlist[0, 0]) - DDry[0,0,:])
ax = axes[1]
ds = calculate_wet_refractivity(t1)
ax.plot(obj2.hgt, ds['N_wet'].isel(time = 0).sel(latitude = obj1.latlist[0, 0], longitude = obj1.lonlist[0, 0]) - DWet[0,0,:])

ax = axes[2]
ds = calculate_refractive_indexes(t1)
ax.plot(obj2.hgt, ds['N'].isel(time = 0).sel(latitude = obj1.latlist[0, 0], longitude = obj1.lonlist[0, 0]) - obj1.Delfn[0,0,:])

# from scipy import integrate as intg
# fig, axes = plt.subplots(3)
# ax = axes[0]
# [DDry,DWet] = pa.processor.PTV2del(obj2.Pi,obj2.Ti,obj2.Vi,obj2.hgt,obj2.dict,verbose=True)
# ds = calculate_dry_refractivity(t2)
# ax.plot(obj2.hgt, ds['N_dry'].isel(time = 0).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]) - DDry[0,0,:])
# ax = axes[1]
# ds = calculate_wet_refractivity(t2)
# ax.plot(obj2.hgt, ds['N_wet'].isel(time = 0).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]) - DWet[0,0,:])

# ax = axes[2]
# ds = calculate_refractive_indexes(t2)
# ax.plot(obj2.hgt, ds['N'].isel(time = 0).sel(latitude = obj2.latlist[0, 0], longitude = obj2.lonlist[0, 0]) - obj2.Delfn[0,0,:])

In [ ]:
dem_ds = dem.to_dataset(name = 'dem')
dem_ds['inc'] = inc
ds1 = calculate_refractive_indexes(t1)
ds1 = get_delay(ds1, dem_ds, wavelength = 0.238403545)

ds2 = calculate_refractive_indexes(t2)
ds2 = get_delay(ds2, dem_ds, wavelength = 0.238403545)

In [ ]:

# phs = obj2.getdelay(wvl = 0.238403545) - obj1.getdelay(wvl = 0.238403545)
delay_diff = (ds2['delay'].isel(time = 0) - ds1['delay'].isel(time = 0))

In [ ]:
fig, axes = plt.subplots(1, 2)
dem.plot(ax = axes[1], vmin = 0, vmax = 1650)
delay_diff.transpose().plot(ax = axes[0])
plt.tight_layout()